# Voice anonymization tutorial

In [4]:
# Download EmoDB dataset (speech emotion recognition dataset comprises 535 recording spoken in german by 10 actors with 7 emotions)
# link: http://emodb.bilderbar.info/index-1280.html

In [10]:
%%bash

dataset_url="http://emodb.bilderbar.info/download/download.zip"
data_folder="../data/"
dataset_name="emodb_dataset"

dataset_path="$data_folder""$dataset_name"
if [ -d "$dataset_path" ]; then
  echo "$dataset_name already downloaded in $dataset_path."
else
  echo "Downloading..."
  mkdir -p "$dataset_path"
  wget -O "$dataset_path"/"$dataset_name".zip "$dataset_url"
  unzip "$dataset_path"/"$dataset_name".zip -d "$dataset_path"
  rm "$dataset_path"/"$dataset_name".zip
fi

Downloading...


--2023-07-12 00:20:33--  http://emodb.bilderbar.info/download/download.zip
Resolving emodb.bilderbar.info (emodb.bilderbar.info)... 85.13.147.80
Connecting to emodb.bilderbar.info (emodb.bilderbar.info)|85.13.147.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40567066 (39M) [application/zip]
Saving to: ‘../data/emodb_dataset/emodb_dataset.zip’

     0K .......... .......... .......... .......... ..........  0%  200K 3m18s
    50K .......... .......... .......... .......... ..........  0%  408K 2m27s
   100K .......... .......... .......... .......... ..........  0% 2.83M 1m42s
   150K .......... .......... .......... .......... ..........  0%  432K 1m40s
   200K .......... .......... .......... .......... ..........  0% 5.75M 81s
   250K .......... .......... .......... .......... ..........  0%  417K 83s
   300K .......... .......... .......... .......... ..........  0% 5.18M 72s
   350K .......... .......... .......... .......... ..........  1%  405K 75s


  5100K .......... .......... .......... .......... .......... 12% 1.26M 30s
  5150K .......... .......... .......... .......... .......... 13%  655K 31s
  5200K .......... .......... .......... .......... .......... 13%  364K 31s
  5250K .......... .......... .......... .......... .......... 13%  215M 31s
  5300K .......... .......... .......... .......... .......... 13%  349M 30s
  5350K .......... .......... .......... .......... .......... 13%  850K 31s
  5400K .......... .......... .......... .......... .......... 13%  688K 31s
  5450K .......... .......... .......... .......... .......... 13%  920K 31s
  5500K .......... .......... .......... .......... .......... 14%  677K 31s
  5550K .......... .......... .......... .......... .......... 14% 1.27M 31s
  5600K .......... .......... .......... .......... .......... 14%  584K 31s
  5650K .......... .......... .......... .......... .......... 14% 1.80M 31s
  5700K .......... .......... .......... .......... .......... 14%  574K 31s

 10450K .......... .......... .......... .......... .......... 26%  923K 29s
 10500K .......... .......... .......... .......... .......... 26%  711K 29s
 10550K .......... .......... .......... .......... .......... 26% 3.64M 29s
 10600K .......... .......... .......... .......... .......... 26%  653K 29s
 10650K .......... .......... .......... .......... .......... 27% 1.30M 29s
 10700K .......... .......... .......... .......... .......... 27%  713K 29s
 10750K .......... .......... .......... .......... .......... 27%  867K 29s
 10800K .......... .......... .......... .......... .......... 27% 5.48M 28s
 10850K .......... .......... .......... .......... .......... 27%  674K 28s
 10900K .......... .......... .......... .......... .......... 27% 1.08M 28s
 10950K .......... .......... .......... .......... .......... 27%  901K 28s
 11000K .......... .......... .......... .......... .......... 27%  809K 28s
 11050K .......... .......... .......... .......... .......... 28% 6.92M 28s

 15800K .......... .......... .......... .......... .......... 40%  107M 24s
 15850K .......... .......... .......... .......... .......... 40% 5.96M 24s
 15900K .......... .......... .......... .......... .......... 40%  418K 24s
 15950K .......... .......... .......... .......... .......... 40%  833K 24s
 16000K .......... .......... .......... .......... .......... 40%  772K 24s
 16050K .......... .......... .......... .......... .......... 40%  484K 24s
 16100K .......... .......... .......... .......... .......... 40% 2.83M 24s
 16150K .......... .......... .......... .......... .......... 40%  425K 24s
 16200K .......... .......... .......... .......... .......... 41% 37.7M 24s
 16250K .......... .......... .......... .......... .......... 41%  415K 24s
 16300K .......... .......... .......... .......... .......... 41% 59.0M 24s
 16350K .......... .......... .......... .......... .......... 41%  406K 24s
 16400K .......... .......... .......... .......... .......... 41% 30.9M 24s

...

  inflating: ../data/emodb_dataset/silb/14a07Ld.silb  
  inflating: ../data/emodb_dataset/silb/15a02Na.silb  
  inflating: ../data/emodb_dataset/silb/08a04La.silb  
  inflating: ../data/emodb_dataset/silb/16a01Fc.silb  
  inflating: ../data/emodb_dataset/silb/14b09Wa.silb  
  inflating: ../data/emodb_dataset/silb/10a07La.silb  
  inflating: ../data/emodb_dataset/silb/14a01Wc.silb  
  inflating: ../data/emodb_dataset/silb/13a05Ea.silb  
  inflating: ../data/emodb_dataset/silb/09b03Wb.silb  
  inflating: ../data/emodb_dataset/silb/15a07Nc.silb  
  inflating: ../data/emodb_dataset/silb/16b09Fb.silb  
  inflating: ../data/emodb_dataset/silb/10a01Wa.silb  
  inflating: ../data/emodb_dataset/silb/14b09Wc.silb  
  inflating: ../data/emodb_dataset/silb/16a01Tb.silb  
  inflating: ../data/emodb_dataset/silb/03b10Na.silb  
  inflating: ../data/emodb_dataset/silb/10a02Lb.silb  
  inflating: ../data/emodb_dataset/silb/12b02Na.silb  
  inflating: ../data/emodb_dataset/silb/14b10Ad.silb  
  inflatin

  inflating: ../data/emodb_dataset/silb/03a02Ta.silb  
  inflating: ../data/emodb_dataset/silb/16b09Ab.silb  
  inflating: ../data/emodb_dataset/silb/16b03La.silb  
  inflating: ../data/emodb_dataset/silb/11b02Na.silb  
  inflating: ../data/emodb_dataset/silb/08b03Wd.silb  
  inflating: ../data/emodb_dataset/silb/12a05Lb.silb  
  inflating: ../data/emodb_dataset/silb/13b01Ab.silb  
  inflating: ../data/emodb_dataset/silb/09b09Nd.silb  
  inflating: ../data/emodb_dataset/silb/15b09Ta.silb  
  inflating: ../data/emodb_dataset/silb/10a02Na.silb  
  inflating: ../data/emodb_dataset/silb/03a05Aa.silb  
  inflating: ../data/emodb_dataset/silb/16b02Wb.silb  
  inflating: ../data/emodb_dataset/silb/12b01Ta.silb  
  inflating: ../data/emodb_dataset/silb/08a04Nc.silb  
  inflating: ../data/emodb_dataset/silb/15a04Ab.silb  
  inflating: ../data/emodb_dataset/silb/12a04Wc.silb  
  inflating: ../data/emodb_dataset/silb/15a02Wb.silb  
  inflating: ../data/emodb_dataset/silb/13a05Nb.silb  
  inflatin

  inflating: ../data/emodb_dataset/wav/09b03Lb.wav  
  inflating: ../data/emodb_dataset/wav/09b02Wc.wav  
  inflating: ../data/emodb_dataset/wav/16a07La.wav  
  inflating: ../data/emodb_dataset/wav/09b02Wd.wav  
  inflating: ../data/emodb_dataset/wav/16a07Lb.wav  
  inflating: ../data/emodb_dataset/wav/16b10Aa.wav  
  inflating: ../data/emodb_dataset/wav/12a07La.wav  
  inflating: ../data/emodb_dataset/wav/03a04Fd.wav  
  inflating: ../data/emodb_dataset/wav/16b09Wb.wav  
  inflating: ../data/emodb_dataset/wav/12b10Ac.wav  
  inflating: ../data/emodb_dataset/wav/12b09Wc.wav  
  inflating: ../data/emodb_dataset/wav/09b10Aa.wav  
  inflating: ../data/emodb_dataset/wav/09b09Wa.wav  
  inflating: ../data/emodb_dataset/wav/11b10Td.wav  
  inflating: ../data/emodb_dataset/wav/16b03Nb.wav  
  inflating: ../data/emodb_dataset/wav/13b02Fb.wav  
  inflating: ../data/emodb_dataset/wav/08b10Tc.wav  
  inflating: ../data/emodb_dataset/wav/13b09Ec.wav  
  inflating: ../data/emodb_dataset/wav/09b03Nb

  inflating: ../data/emodb_dataset/wav/12b10Wa.wav  
  inflating: ../data/emodb_dataset/wav/16a02Nb.wav  
  inflating: ../data/emodb_dataset/wav/09b10Wa.wav  
  inflating: ../data/emodb_dataset/wav/12a02Nb.wav  
  inflating: ../data/emodb_dataset/wav/13a01Fd.wav  
  inflating: ../data/emodb_dataset/wav/11b01Ab.wav  
  inflating: ../data/emodb_dataset/wav/08b01Aa.wav  
  inflating: ../data/emodb_dataset/wav/13b10Ec.wav  
  inflating: ../data/emodb_dataset/wav/14a05Ta.wav  
  inflating: ../data/emodb_dataset/wav/14a05Tc.wav  
  inflating: ../data/emodb_dataset/wav/03b01Td.wav  
  inflating: ../data/emodb_dataset/wav/13b10Fa.wav  
  inflating: ../data/emodb_dataset/wav/11a05Ad.wav  
  inflating: ../data/emodb_dataset/wav/10a05Tb.wav  
  inflating: ../data/emodb_dataset/wav/08a04La.wav  
  inflating: ../data/emodb_dataset/wav/03a05Tc.wav  
  inflating: ../data/emodb_dataset/wav/15a04Nc.wav  
  inflating: ../data/emodb_dataset/wav/14b01Wc.wav  
  inflating: ../data/emodb_dataset/wav/14b03Ad

  inflating: ../data/emodb_dataset/wav/08b01Wa.wav  
  inflating: ../data/emodb_dataset/wav/16b01Eb.wav  
  inflating: ../data/emodb_dataset/wav/15a05Wa.wav  
  inflating: ../data/emodb_dataset/wav/15a07Ac.wav  
  inflating: ../data/emodb_dataset/wav/15b09La.wav  
  inflating: ../data/emodb_dataset/wav/14a07Tc.wav  
  inflating: ../data/emodb_dataset/wav/03b03Tc.wav  
  inflating: ../data/emodb_dataset/wav/11a07Ac.wav  
  inflating: ../data/emodb_dataset/wav/10a07Ta.wav  
  inflating: ../data/emodb_dataset/wav/09b01Ea.wav  
  inflating: ../data/emodb_dataset/wav/11a05Wd.wav  
  inflating: ../data/emodb_dataset/wav/08a05Wa.wav  
  inflating: ../data/emodb_dataset/wav/16a05Ea.wav  
  inflating: ../data/emodb_dataset/wav/11b09Ld.wav  
  inflating: ../data/emodb_dataset/wav/16b01Fa.wav  
  inflating: ../data/emodb_dataset/wav/08b09Lc.wav  
  inflating: ../data/emodb_dataset/wav/15b02Nd.wav  
  inflating: ../data/emodb_dataset/wav/11b02Na.wav  
  inflating: ../data/emodb_dataset/wav/09a05Ed

In [33]:
import os
import pandas as pd 
import numpy as np
import torchaudio
import torch

In [16]:
folder_path = "../data/emodb_dataset/wav/" 

# TODO: CHECK IF I CAN REUSE VARIABLES ABOVE!!!!!!




file_details = []
for file_name in os.listdir(folder_path):
    if file_name.endswith(".wav"):
        file_path = os.path.join(folder_path, file_name)
        speaker = file_name[:2]
        file_details.append([file_path, file_name, speaker])

df = pd.DataFrame(file_details, columns=["Path", "Name", "Speaker"])

In [17]:
df

,Path,Name,Speaker
0,../data/emodb_dataset/wav/16a02Lb.wav,16a02Lb.wav,16
1,../data/emodb_dataset/wav/14a07Wc.wav,14a07Wc.wav,14
2,../data/emodb_dataset/wav/10a07Ad.wav,10a07Ad.wav,10
3,../data/emodb_dataset/wav/13a05Ea.wav,13a05Ea.wav,13
4,../data/emodb_dataset/wav/14a05Wa.wav,14a05Wa.wav,14
...,...,...,...
530,../data/emodb_dataset/wav/08b01Fd.wav,08b01Fd.wav,08
531,../data/emodb_dataset/wav/03b02Aa.wav,03b02Aa.wav,03
532,../data/emodb_dataset/wav/09a05Lc.wav,09a05Lc.wav,09
533,../data/emodb_dataset/wav/08a05Lc.wav,08a05Lc.wav,08


In [19]:
def load_audio(file_path):
    waveform, sample_rate = torchaudio.load(file_path)
    return waveform

In [38]:
for index, row in df.iterrows():
    file_path = row["Path"]
    waveform = load_audio(file_path)
    df.at[index, "Waveform"] = waveform.squeeze()

In [39]:
df

,Path,Name,Speaker,Waveform
0,../data/emodb_dataset/wav/16a02Lb.wav,16a02Lb.wav,16,"[tensor(0.0013), tensor(0.0008), tensor(0.0005..."
1,../data/emodb_dataset/wav/14a07Wc.wav,14a07Wc.wav,14,"[tensor(0.), tensor(0.), tensor(-3.0518e-05), ..."
2,../data/emodb_dataset/wav/10a07Ad.wav,10a07Ad.wav,10,"[tensor(0.0012), tensor(0.0010), tensor(0.), t..."
3,../data/emodb_dataset/wav/13a05Ea.wav,13a05Ea.wav,13,"[tensor(0.0016), tensor(0.0002), tensor(-0.000..."
4,../data/emodb_dataset/wav/14a05Wa.wav,14a05Wa.wav,14,"[tensor(0.0003), tensor(0.0002), tensor(-6.103..."
...,...,...,...,...
530,../data/emodb_dataset/wav/08b01Fd.wav,08b01Fd.wav,08,"[tensor(0.0012), tensor(0.0013), tensor(0.0004..."
531,../data/emodb_dataset/wav/03b02Aa.wav,03b02Aa.wav,03,"[tensor(0.), tensor(-0.0004), tensor(0.0002), ..."
532,../data/emodb_dataset/wav/09a05Lc.wav,09a05Lc.wav,09,"[tensor(-0.0452), tensor(-0.0444), tensor(-0.0..."
533,../data/emodb_dataset/wav/08a05Lc.wav,08a05Lc.wav,08,"[tensor(-0.0021), tensor(-0.0008), tensor(-0.0..."


In [ ]:
df["Embeddings"]

df_anonymized["Path"]
df_anonymized["Name"]
df_anonymized["Speaker"]
df_anonymized["Signal"]
df_anonymized["Embeddings"]